# Part V — APIs with Django  
## 25. DRF Advanced Patterns (Nested Writes, Filtering, Throttling, Uploads, Performance, Error Shaping)

This chapter is where your DRF API starts looking like a real production API:

- **Writable nested relationships** (e.g., article tags by slug)
- **Object-level permissions** with clear policies
- **django-filter** for consistent filtering
- **Ordering + search** with whitelists
- **Pagination** customization (page size caps, cursor pagination concepts)
- **File uploads** via multipart
- **Performance**: avoid serializer-driven N+1 queries
- **Throttling** (rate limiting) to reduce abuse
- **Consistent error responses** using a custom exception handler
- **API testing patterns** for correctness + performance regressions

You’ll apply these to the API resources you already have:
- Articles (public read, authenticated write by author/staff)
- Tasks (org-scoped, role rules)
- Comments (nested under articles; moderation-aware)

---

## 25.0 Learning Outcomes

By the end, you should be able to:

1. Build serializers that support **both read and write** cleanly (different shapes).
2. Implement **writable nested relations** safely and transactionally.
3. Implement filtering via `django-filter` (`FilterSet`) instead of manual parsing.
4. Add search and ordering safely (whitelisted fields).
5. Customize pagination (page_size query param with caps; cursor pagination option).
6. Implement file upload endpoints (multipart) with validation.
7. Make serializers fast by designing querysets and prefetching correctly.
8. Add throttling and test 429 behavior.
9. Produce consistent API error shapes using a custom exception handler.
10. Write DRF tests for:
   - permissions
   - validation errors
   - filtering/pagination
   - file uploads
   - query counts (performance regression tests)

---

## 25.1 Serializer Design at Scale (Read vs Write Shapes)

A professional DRF codebase almost always separates:

- **Read serializer** (more fields, nested, computed, display-friendly)
- **Write serializer** (accepts inputs, validates, usually uses IDs/slugs)

Why:
- Read models can be rich and convenient for clients.
- Write models must be strict and secure (never accept fields like `author`,
  `organization`, `created_by` from the client).

### 25.1.1 Example: Article read vs write

**Read (detail) should return:**
- author object
- tags as objects
- cover_image URL
- body

**Write should accept:**
- title, body
- status (optional policy)
- tags by slug list (or tag IDs)
- cover_image (multipart upload or separate endpoint)

---

## 25.2 Writable Nested Relations: Tags by Slug (Safe Pattern)

Goal: allow clients to create/update an article with:

```json
{
  "title": "Hello",
  "slug": "hello",
  "body": "Body...",
  "tags": ["django", "routing"]
}
```

### 25.2.1 Create a “tag slug list” field in serializer

Create `articles/api/serializers_write.py`:

```python
from __future__ import annotations

from django.db import transaction
from django.utils.text import slugify
from rest_framework import serializers

from articles.models import Article, Tag


class ArticleWriteSerializer(serializers.ModelSerializer):
    tags = serializers.ListField(
        child=serializers.CharField(max_length=60),
        required=False,
        allow_empty=True,
    )

    class Meta:
        model = Article
        fields = [
            "title",
            "slug",
            "body",
            "status",
            "published_at",
            "tags",
            "cover_image",
        ]
        extra_kwargs = {
            "published_at": {"required": False, "allow_null": True},
            "cover_image": {"required": False, "allow_null": True},
        }

    def validate_slug(self, value: str) -> str:
        value = slugify((value or "").strip())
        if not value:
            raise serializers.ValidationError("Slug is required.")
        return value

    def validate_tags(self, value: list[str]) -> list[str]:
        normalized: list[str] = []
        for raw in value:
            slug = slugify((raw or "").strip())
            if not slug:
                raise serializers.ValidationError("Tags must be non-empty slugs.")
            normalized.append(slug)

        # Remove duplicates while preserving order
        seen: set[str] = set()
        deduped: list[str] = []
        for s in normalized:
            if s not in seen:
                seen.add(s)
                deduped.append(s)

        if len(deduped) > 20:
            raise serializers.ValidationError("Too many tags (max 20).")

        return deduped

    @transaction.atomic
    def create(self, validated_data):
        request = self.context["request"]
        tag_slugs = validated_data.pop("tags", [])

        article = Article(**validated_data)
        article.author = request.user
        article.save()

        tags = self._get_or_create_tags(tag_slugs)
        article.tags.set(tags)

        return article

    @transaction.atomic
    def update(self, instance: Article, validated_data):
        tag_slugs = validated_data.pop("tags", None)

        for field, value in validated_data.items():
            setattr(instance, field, value)

        instance.save()

        if tag_slugs is not None:
            tags = self._get_or_create_tags(tag_slugs)
            instance.tags.set(tags)

        return instance

    def _get_or_create_tags(self, slugs: list[str]) -> list[Tag]:
        tags: list[Tag] = []
        for slug in slugs:
            tag, _ = Tag.objects.get_or_create(
                slug=slug,
                defaults={"name": slug.replace("-", " ").title()},
            )
            tags.append(tag)
        return tags
```

### 25.2.2 Why this is “safe and professional”

- We do **not** accept `author` from client; we set it from `request.user`.
- We normalize slugs (`slugify`) so clients can send `"Django"` and it becomes
  `"django"`.
- We run the operation inside `transaction.atomic()`:
  - either the article + tags relation is fully updated, or nothing is.
- We cap tag count to avoid abuse.

### 25.2.3 Policy decision: should API allow setting `status`?
You must decide:
- Public users can only create drafts; publishing is staff-only.
- Or authors can publish their own content.

A common policy:

- allow authenticated users to create drafts only via API
- only staff can publish (or require special permission)

We’ll implement that policy in a validation step later.

---

## 25.3 Article API: Object-Level Permissions + Write Support

### 25.3.1 Permission: author or staff can edit; anyone can read published

Create `articles/api/permissions.py`:

```python
from rest_framework.permissions import BasePermission, SAFE_METHODS

from articles.models import Article


class IsAuthorOrStaffOrReadOnly(BasePermission):
    def has_object_permission(self, request, view, obj: Article) -> bool:
        if request.method in SAFE_METHODS:
            return True
        if not request.user.is_authenticated:
            return False
        if request.user.is_staff:
            return True
        return obj.author_id == request.user.id
```

### 25.3.2 Update ArticleViewSet to support writes
In `articles/api/views.py`:

```python
from django.db.models import Q
from rest_framework import permissions, viewsets

from articles.api.permissions import IsAuthorOrStaffOrReadOnly
from articles.api.serializers import (
    ArticleDetailSerializer,
    ArticleListSerializer,
)
from articles.api.serializers_write import ArticleWriteSerializer
from articles.models import Article


class ArticleViewSet(viewsets.ModelViewSet):
    lookup_field = "slug"
    permission_classes = [IsAuthorOrStaffOrReadOnly]

    def get_queryset(self):
        user = self.request.user

        if self.action in {"list", "retrieve"}:
            qs = (
                Article.objects.published()
                .select_related("author")
                .prefetch_related("tags")
            )
        else:
            # For write operations we must allow authors/staff to access drafts too.
            qs = (
                Article.objects.visible_to(user)
                .select_related("author")
                .prefetch_related("tags")
            )

        q = (self.request.query_params.get("q") or "").strip()
        tag = (self.request.query_params.get("tag") or "").strip()

        if tag:
            qs = qs.filter(tags__slug=tag)

        if q:
            qs = qs.filter(Q(title__icontains=q) | Q(body__icontains=q))

        return qs.distinct().order_by("-published_at", "-created_at")

    def get_serializer_class(self):
        if self.action in {"create", "update", "partial_update"}:
            return ArticleWriteSerializer
        if self.action == "retrieve":
            return ArticleDetailSerializer
        return ArticleListSerializer

    def get_permissions(self):
        # AllowAny for safe methods; writes require auth.
        if self.request.method in permissions.SAFE_METHODS:
            return [permissions.AllowAny()]
        return [permissions.IsAuthenticated(), IsAuthorOrStaffOrReadOnly()]
```

### 25.3.3 Add a publish policy: only staff can set status=published via API
Add to `ArticleWriteSerializer.validate()`:

```python
def validate(self, attrs):
    request = self.context["request"]
    status = attrs.get("status")

    if status == Article.Status.PUBLISHED and not request.user.is_staff:
        raise serializers.ValidationError(
            {"status": "Only staff can publish via API."}
        )

    return attrs
```

This prevents “normal users publish everything” through the API even if your HTML
site had different rules.

---

## 25.4 django-filter (Real Filtering Without Manual Parsing)

Manual query param parsing works, but becomes repetitive and inconsistent. The
professional standard in DRF is:

- `django-filter` + `DjangoFilterBackend` + `FilterSet`

### 25.4.1 Enable filter backend in settings
In `config/settings.py`:

```python
REST_FRAMEWORK = {
    # ...
    "DEFAULT_FILTER_BACKENDS": [
        "django_filters.rest_framework.DjangoFilterBackend",
        "rest_framework.filters.SearchFilter",
        "rest_framework.filters.OrderingFilter",
    ],
}
```

Make sure `django_filters` is installed and in `INSTALLED_APPS`:

```python
INSTALLED_APPS += ["django_filters"]
```

### 25.4.2 Article FilterSet
Create `articles/api/filters.py`:

```python
import django_filters

from articles.models import Article


class ArticleFilter(django_filters.FilterSet):
    tag = django_filters.CharFilter(field_name="tags__slug")
    status = django_filters.ChoiceFilter(choices=Article.Status.choices)

    class Meta:
        model = Article
        fields = ["tag", "status"]
```

Update `ArticleViewSet`:

```python
from articles.api.filters import ArticleFilter

class ArticleViewSet(viewsets.ModelViewSet):
    filterset_class = ArticleFilter
    search_fields = ["title", "body"]
    ordering_fields = ["published_at", "created_at", "title"]
    ordering = ["-published_at", "-created_at"]
```

Now clients can do:

- `/api/v1/articles/?tag=django`
- `/api/v1/articles/?search=templates`
- `/api/v1/articles/?ordering=-created_at`

**Why `search` instead of `q`?**  
DRF’s `SearchFilter` uses `?search=...`. You can keep your `q` too, but using DRF’s
standard reduces custom code. Many teams choose DRF defaults.

If you want to accept `q`, you can:
- keep manual `q` parsing, or
- write a custom filter backend, or
- document `search` and use that.

Industry preference: use DRF’s `search` and document it.

---

## 25.5 Advanced Pagination (Page Size Param + Caps, Cursor Option)

### 25.5.1 Custom PageNumberPagination with `page_size` param
Create `config/api_pagination.py`:

```python
from rest_framework.pagination import PageNumberPagination


class StandardResultsPagination(PageNumberPagination):
    page_size = 20
    page_size_query_param = "page_size"
    max_page_size = 100
```

In settings:

```python
REST_FRAMEWORK = {
    # ...
    "DEFAULT_PAGINATION_CLASS": "config.api_pagination.StandardResultsPagination",
}
```

Now:
- `/api/v1/articles/?page_size=50` works
- but capped at 100 (DoS protection)

### 25.5.2 Cursor pagination (when to use)
DRF supports `CursorPagination`. Use it when:
- deep pagination is common
- offset pagination is too slow
- you need stable paging under inserts

Example (optional):

```python
from rest_framework.pagination import CursorPagination


class ArticleCursorPagination(CursorPagination):
    page_size = 20
    ordering = "-published_at"
```

Then set on the viewset:

```python
pagination_class = ArticleCursorPagination
```

Cursor pagination changes response style and navigation, so treat it as a deliberate
API design choice.

---

## 25.6 File Uploads in DRF (Multipart + Validation)

You already upload cover images in HTML forms. Now support it via API.

### 25.6.1 Common upload strategies
1. Upload as part of article create/update (multipart request).
2. Separate endpoint `/api/v1/articles/<slug>/cover/` for uploading just the file.

We’ll implement **strategy 2** because it’s clearer and often used in real apps.

### 25.6.2 Create an upload serializer
`articles/api/serializers_upload.py`:

```python
from rest_framework import serializers

from articles.models import Article


class ArticleCoverUploadSerializer(serializers.ModelSerializer):
    class Meta:
        model = Article
        fields = ["cover_image"]

    def validate_cover_image(self, image):
        if not image:
            return image

        max_bytes = 2 * 1024 * 1024
        if image.size > max_bytes:
            raise serializers.ValidationError("Cover image too large (max 2 MB).")

        allowed = {"image/jpeg", "image/png", "image/webp"}
        content_type = getattr(image, "content_type", None)
        if content_type and content_type not in allowed:
            raise serializers.ValidationError("Unsupported image type.")

        return image
```

### 25.6.3 Create a custom action on ArticleViewSet
In `articles/api/views.py`:

```python
from rest_framework.decorators import action
from rest_framework.parsers import FormParser, MultiPartParser
from rest_framework.response import Response
from rest_framework import status

from articles.api.serializers_upload import ArticleCoverUploadSerializer


class ArticleViewSet(viewsets.ModelViewSet):
    # ...

    @action(
        detail=True,
        methods=["post"],
        url_path="cover",
        parser_classes=[MultiPartParser, FormParser],
    )
    def upload_cover(self, request, slug=None):
        article = self.get_object()

        serializer = ArticleCoverUploadSerializer(
            instance=article,
            data=request.data,
            partial=True,
            context={"request": request},
        )
        serializer.is_valid(raise_exception=True)
        serializer.save()

        return Response(
            {
                "status": "ok",
                "cover_image": article.cover_image.url
                if article.cover_image
                else None,
            },
            status=status.HTTP_200_OK,
        )
```

### 25.6.4 How to call it (curl)
Assuming session auth and you’re logged in in browser, curl is awkward. For a token
auth API client, it’s easy. Still, you can test multipart format:

```bash
curl -i -X POST \
  -F "cover_image=@/path/to/image.png" \
  http://127.0.0.1:8000/api/v1/articles/hello-django/cover/
```

If you keep SessionAuthentication, this POST will likely require CSRF and a session
cookie. For now, test via browsable API or switch to token auth in a later chapter.

---

## 25.7 Throttling (Rate Limiting) in DRF

Throttling reduces abuse and protects availability. Common targets:
- login endpoints (if you expose API auth)
- comment submission
- expensive searches
- file uploads

### 25.7.1 Enable global throttling
In settings:

```python
REST_FRAMEWORK = {
    # ...
    "DEFAULT_THROTTLE_CLASSES": [
        "rest_framework.throttling.AnonRateThrottle",
        "rest_framework.throttling.UserRateThrottle",
    ],
    "DEFAULT_THROTTLE_RATES": {
        "anon": "60/min",
        "user": "600/min",
    },
}
```

### 25.7.2 Per-endpoint throttling (scoped)
Example: comment create should be tighter.

Create a throttle:

```python
from rest_framework.throttling import ScopedRateThrottle

class CommentThrottle(ScopedRateThrottle):
    scope = "comments"
```

In settings:

```python
"DEFAULT_THROTTLE_RATES": {
    "anon": "60/min",
    "user": "600/min",
    "comments": "10/min",
}
```

Then on a view/action:

```python
throttle_classes = [CommentThrottle]
```

### 25.7.3 Testing throttling
In tests you can override throttles to a very low rate and assert 429:

```python
from django.test import override_settings
from rest_framework.test import APITestCase


@override_settings(
    REST_FRAMEWORK={
        "DEFAULT_THROTTLE_CLASSES": [
            "rest_framework.throttling.AnonRateThrottle"
        ],
        "DEFAULT_THROTTLE_RATES": {"anon": "1/min"},
    }
)
class ThrottleTests(APITestCase):
    def test_throttle_hits_429(self):
        url = "/api/v1/articles/"
        r1 = self.client.get(url)
        self.assertEqual(r1.status_code, 200)

        r2 = self.client.get(url)
        self.assertEqual(r2.status_code, 429)
```

---

## 25.8 Performance: Stop Serializer-Driven N+1 Queries

DRF serializers can accidentally create N+1 problems.

### 25.8.1 The most common pitfall: per-object DB work in SerializerMethodField

Bad pattern:

```python
class ArticleListSerializer(serializers.ModelSerializer):
    tag_count = serializers.SerializerMethodField()

    def get_tag_count(self, obj):
        return obj.tags.count()  # can become N+1
```

Fix patterns:
1. annotate in queryset:

```python
from django.db.models import Count

qs = Article.objects.published().annotate(
    tag_count=Count("tags", distinct=True)
)
```

Then serializer uses integer field:

```python
tag_count = serializers.IntegerField(read_only=True)
```

2. or prefetch tags and compute in Python (still extra work but no extra queries):

```python
return len(obj.tags.all())
```

But annotation is usually better for counts.

### 25.8.2 Always design queryset to match serializer needs
If serializer returns:
- author fields → `select_related("author")`
- tags → `prefetch_related("tags")`
- approved comments → `Prefetch(...)` filtered

Example approved comments prefetch:

```python
from django.db.models import Prefetch
from articles.models import Comment

qs = Article.objects.published().prefetch_related(
    Prefetch(
        "comments",
        queryset=Comment.objects.filter(is_approved=True).order_by("created_at"),
        to_attr="approved_comments",
    )
)
```

Then serializer uses `obj.approved_comments` without DB hits.

---

## 25.9 Consistent API Error Shaping (Custom Exception Handler)

DRF default error format is fine, but many teams want a consistent envelope:

```json
{
  "error": {
    "code": "validation_error",
    "message": "Your request is invalid.",
    "details": { ... }
  }
}
```

### 25.9.1 Implement a custom exception handler
Create `config/drf_exceptions.py`:

```python
from rest_framework.views import exception_handler
from rest_framework.response import Response
from rest_framework import status


def custom_exception_handler(exc, context):
    response = exception_handler(exc, context)
    if response is None:
        return response

    if response.status_code == status.HTTP_400_BAD_REQUEST:
        code = "validation_error"
        message = "Your request is invalid."
    elif response.status_code == status.HTTP_401_UNAUTHORIZED:
        code = "not_authenticated"
        message = "Authentication required."
    elif response.status_code == status.HTTP_403_FORBIDDEN:
        code = "permission_denied"
        message = "You do not have permission."
    elif response.status_code == status.HTTP_404_NOT_FOUND:
        code = "not_found"
        message = "Not found."
    elif response.status_code == status.HTTP_429_TOO_MANY_REQUESTS:
        code = "rate_limited"
        message = "Too many requests."
    else:
        code = "error"
        message = "Request failed."

    details = response.data

    response.data = {
        "error": {
            "code": code,
            "message": message,
            "details": details,
        }
    }
    return response
```

Enable in settings:

```python
REST_FRAMEWORK = {
    # ...
    "EXCEPTION_HANDLER": "config.drf_exceptions.custom_exception_handler",
}
```

Now all DRF errors become consistent while still preserving field-level details.

---

## 25.10 Nested Endpoints: Comments Under Articles (DRF Style)

We’ll implement:

- `GET /api/v1/articles/<slug>/comments/` (approved only)
- `POST /api/v1/articles/<slug>/comments/` (create; unapproved)

### 25.10.1 Serializer
`articles/api/comments_serializers.py`:

```python
from rest_framework import serializers

from articles.models import Comment


class CommentSerializer(serializers.ModelSerializer):
    class Meta:
        model = Comment
        fields = ["id", "name", "body", "created_at"]
        read_only_fields = ["id", "created_at"]
```

### 25.10.2 View
`articles/api/comments_views.py`:

```python
from rest_framework import permissions
from rest_framework.response import Response
from rest_framework.views import APIView
from rest_framework import status

from articles.models import Article
from articles.api.comments_serializers import CommentSerializer


class ArticleCommentsView(APIView):
    permission_classes = [permissions.AllowAny]

    def get(self, request, slug: str):
        article = Article.objects.published().get(slug=slug)
        qs = article.comments.filter(is_approved=True).order_by("created_at")
        data = CommentSerializer(qs, many=True).data
        return Response({"results": data})

    def post(self, request, slug: str):
        article = Article.objects.published().get(slug=slug)

        serializer = CommentSerializer(data=request.data)
        serializer.is_valid(raise_exception=True)

        comment = serializer.save(article=article, is_approved=False)

        return Response(
            {
                "status": "submitted",
                "comment": CommentSerializer(comment).data,
            },
            status=status.HTTP_201_CREATED,
        )
```

Add URL in `articles/api/urls.py`:

```python
from django.urls import include, path
from rest_framework.routers import DefaultRouter

from articles.api.views import ArticleViewSet
from articles.api.comments_views import ArticleCommentsView

router = DefaultRouter()
router.register("articles", ArticleViewSet, basename="article")

urlpatterns = [
    path("", include(router.urls)),
    path(
        "articles/<slug:slug>/comments/",
        ArticleCommentsView.as_view(),
        name="article_comments",
    ),
]
```

**Important note:** Here we used `.get(...)` which raises `DoesNotExist`. DRF’s
exception handler will convert it to 500 unless you catch and raise 404. Replace
with `get_object_or_404` style:

```python
from django.shortcuts import get_object_or_404
article = get_object_or_404(Article.objects.published(), slug=slug)
```

That’s the correct version in production.

---

## 25.11 Advanced API Testing Patterns

### 25.11.1 Test validation envelope
If you enabled the custom exception handler, tests should reflect it:

```python
from rest_framework.test import APITestCase


class ArticleApiErrorTests(APITestCase):
    def test_invalid_page_size_error_shape(self):
        r = self.client.get("/api/v1/articles/?page_size=999999")
        self.assertIn(r.status_code, {400, 200})
        # If pagination caps, it might be 200. If you reject, it's 400.
        # Choose your policy and test it consistently.
```

### 25.11.2 Test file upload (multipart)
Example (requires authentication as author/staff per your policy):

```python
from django.core.files.uploadedfile import SimpleUploadedFile
from django.utils import timezone
from rest_framework.test import APITestCase

from articles.models import Article
from tests.factories import ArticleFactory, UserFactory


class ArticleUploadApiTests(APITestCase):
    def setUp(self):
        self.user = UserFactory(username="u1", is_staff=True)
        self.client.login(username="u1", password="pass12345")

        self.article = ArticleFactory(
            status=Article.Status.PUBLISHED,
            published_at=timezone.now(),
            author=self.user,
        )

    def test_upload_cover(self):
        file = SimpleUploadedFile(
            "cover.png",
            b"\x89PNG\r\n\x1a\n" + b"fakepngdata",
            content_type="image/png",
        )

        url = f"/api/v1/articles/{self.article.slug}/cover/"
        r = self.client.post(url, data={"cover_image": file}, format="multipart")
        self.assertEqual(r.status_code, 200)
```

### 25.11.3 Performance regression test for API list
Use query count assertions around API list endpoints, because DRF serialization can
trigger extra queries if you misconfigure prefetching.

If you use pytest:

```python
from django.urls import reverse
from django.utils import timezone

from articles.models import Article
from tests.factories import ArticleFactory, TagFactory


def test_articles_api_list_query_count(
    client,
    django_assert_num_queries,
    db,
):
    tag = TagFactory(slug="django")

    for _ in range(20):
        a = ArticleFactory(
            status=Article.Status.PUBLISHED,
            published_at=timezone.now(),
        )
        a.tags.add(tag)

    url = reverse("article-list")
    with django_assert_num_queries(3):
        r = client.get(url)
        assert r.status_code == 200
```

Adjust query count to your middleware/auth stack; the key is “small and stable.”

---

## 25.12 Exercises (Do These Before Proceeding)

1. **Replace manual `q/tag` parsing with DRF filter/search**:
   - Use `SearchFilter` and `django-filter`.
   - Document: clients use `?search=...` and `?tag=...`.

2. **Implement “only staff can publish via API”** and test:
   - non-staff PATCH setting status=published → 400 with validation envelope
   - staff PATCH works

3. **Task API: add filters using django-filter**:
   - `status`, `priority`, `assigned_to=me`
   - prove filtering works with tests

4. **Add throttling to comment POST**:
   - `comments: 3/min`
   - write a test that hits the limit and gets 429

5. **Add a “querystring builder” utility** for pagination links in API responses:
   - ensure page changes preserve tag/search/ordering

---

Next chapter: **Part V — 26. API Security and Production Concerns**  
We’ll cover CORS, token/JWT auth strategies, rate limiting approaches, API versioning
policies, secrets/key rotation, and production-grade API documentation (OpenAPI),
plus hardening against abuse and data leaks.